# Introduction

Welcome to the P1 Workspace with Group 14:
Will Mitchell
Trevor Gordon


Below are some prelimnary Helper methods that benifit the program as a whole with the starting code parameters

In [379]:

# Parameters
GROUP_ID = 'Group14'
ALGORITHM = 'bt'
PUZZLE_TYPE = 'easy'
PUZZLE_PATH = 'puzzles/Easy-P1.txt'

In [380]:
#imports
import random
import math
import matplotlib.pyplot as plt
import numpy as np
from pymongo.message import MAX_INT32


In [381]:
def algorithm_setup_function(PUZZLE_PATH, PUZZLE_TYPE, algorithm):
    #all algorithms below when this functions runs will print their respective results below with a generated graph
    match algorithm:
        case 'bt':
            if(PUZZLE_TYPE == 'easy'):
                print('run bt algo here')
            else:
                print('backtracking only allowed with easy PUZZLE_TYPE')

        case 'fc':
            print('run fc algo here')

        case 'ac3':
            print('run ac3 algo here')

        case 'sa':
            #for a general standard, 10000 iterations generates the appropriate trend increase if you would like
            result_node =  simulated_annealing(10000)
            parse_to_file(result_node.matrix)
        case 'ga':
            print('run ga algo here')

        case _:
            raise Exception('Invalid algorithm')

In [ ]:
def parse_to_file(matrix):
    #remember naming conventions and output file locations


# Helper Functions


In [382]:
def parse_matrix(file_path: str):
    with open(file_path, mode="r") as file:
        new_matrix = []
        for line in file:
            new_line_array = line.replace("\n", "").replace("\ufeff", "").split(",")
            new_matrix.append(new_line_array)

    file.close()
    new_matrix[0][0] = str(new_matrix[0][0])[-1:]
    return new_matrix


parse_matrix(PUZZLE_PATH)



[['?', '?', '8', '?', '5', '6', '?', '?', '?'],
 ['7', '?', '4', '?', '?', '?', '6', '1', '9'],
 ['?', '?', '?', '?', '?', '?', '8', '5', '?'],
 ['6', '?', '7', '?', '2', '9', '5', '?', '?'],
 ['?', '?', '9', '?', '6', '?', '1', '?', '?'],
 ['?', '?', '2', '3', '1', '?', '9', '?', '4'],
 ['?', '3', '5', '?', '?', '?', '?', '?', '?'],
 ['4', '2', '1', '?', '?', '?', '3', '?', '6'],
 ['?', '?', '?', '8', '3', '?', '4', '?', '?']]

In [373]:
def get_square_values(matrix_state, cord_x, cord_y):
    #first row box
    matrix_x_range = ()
    matrix_y_range = ()
    if cord_x <= 2 and cord_y <= 2:
        matrix_x_range = (0, 2)
        matrix_y_range = (0, 2)
    elif (2 < cord_x <= 5) and (cord_y <= 2):
        matrix_x_range = (3, 5)
        matrix_y_range = (0, 2)

    elif (5 < cord_x <= 8) and (cord_y <= 2):
        matrix_x_range = (6, 8)
        matrix_y_range = (0, 2)

    #second row boxes
    elif cord_x <= 2 and 2 < cord_y <= 5:
        matrix_x_range = (0, 2)
        matrix_y_range = (3, 5)

    elif (2 < cord_x <= 5) and 2 < cord_y <= 5:
        matrix_x_range = (3, 5)
        matrix_y_range = (3, 5)

    elif (5 < cord_x <= 8) and 2 < cord_y <= 5:
        matrix_x_range = (6, 8)
        matrix_y_range = (3, 5)


    #third row boxes

    elif cord_x <= 2 and 5 < cord_y <= 8:
        matrix_x_range = (0, 2)
        matrix_y_range = (6, 8)

    elif (2 < cord_x <= 5) and 5 < cord_y <= 8:
        matrix_x_range = (3, 5)
        matrix_y_range = (6, 8)


    elif (5 < cord_x <= 8) and 5 < cord_y <= 8:
        matrix_x_range = (6, 8)
        matrix_y_range = (6, 8)

    square_values = []
    for y in range(matrix_y_range[0], matrix_y_range[1] + 1):
        for x in range(matrix_x_range[0], matrix_x_range[1] + 1):
            if (cord_y, cord_x) != (y, x):
                square_values.append(matrix_state[y][x])

    return square_values


#testing functionality, try it out!
print(get_square_values(parse_matrix(PUZZLE_PATH), 5, 5))


['?', '2', '9', '?', '6', '?', '3', '1']


In [350]:
def get_row_values(sudoku_state, cord_x, cord_y):
    #chose to keep both parameters just for normality and to keep our alldiff functions flow well and read well
    temp_row = []
    for row_value in range(0, 9):
        if (cord_y, row_value) != (cord_y, cord_x):
            temp_row.append(sudoku_state[cord_y][row_value])
    return temp_row


#try out the function
print(get_row_values(parse_matrix(PUZZLE_PATH), 1, 1))

['7', '4', '?', '?', '?', '6', '1', '9']


In [351]:
def get_column_values(sudoku_state, cord_x, cord_y):
    #chose to keep both parameters just for normality and to keep our alldiff functions flow well and read well
    column_values = []
    for col in range(0, 9):
        if col != cord_y:
            #we don't want to add our actual value we are looking at to list
            column_values.append(sudoku_state[col][cord_x])
    return column_values
    #because range is not inclusive


print(get_column_values(parse_matrix(PUZZLE_PATH), 8, 1))

['?', '?', '?', '?', '4', '?', '6', '?']


In [352]:
def alldiff(sudoku_state, cord_x, cord_y):
    #we are assuming the location of this point is NOT a hardcoded value, we will handle search validation elsewhere
    num_conflicts = 0
    #we want to cast as int, to tallying conflicts of hardcoded values to our total conflicts
    space_value = int(sudoku_state[cord_y][cord_x])
    row_values = get_row_values(sudoku_state, cord_x, cord_y)
    column_values = get_column_values(sudoku_state, cord_x, cord_y)
    square_values = get_square_values(sudoku_state, cord_x, cord_y)

    for value in row_values:
        if space_value == int(value):  # Cast to int here
            num_conflicts += 1

    for value in column_values:
        if space_value == int(value):
            num_conflicts += 1

    for value in square_values:
        if space_value == int(value):
            num_conflicts += 1

    return num_conflicts

In [383]:
def get_row_values_back(sudoku_state, row, col):
  my_set = set(sudoku_state[row])
  return my_set

# print(get_row_values(parse_matrix(PUZZLE_PATH),0,0))

In [384]:
def get_col_values_back(sudoku_state, row, col):
  my_set = set()
  for i in range(9):
      my_set.add(sudoku_state[i][col])


  return my_set

print(get_col_values_back(parse_matrix(PUZZLE_PATH),0,0))

{'6', '4', '?', '7'}


In [385]:
def get_square_values_back(sudoku_state, row, col):
    #top left box
    if col <= 2 and row <= 2:
        col_indexes = [0,1,2]
        row_indexes = [0,1,2]

    #top middle box
    elif (2 < col <= 5) and (row <= 2):
        col_indexes = [3,4,5]
        row_indexes = [0,1,2]

    #top right box
    elif (5 < col <= 8) and (row <= 2):
        col_indexes = [6,7,8]
        row_indexes = [0,1,2]

    #middle left box
    elif (col <= 2) and (2 < row <= 5):
        col_indexes = [0,1,2]
        row_indexes = [3,4,5]

    #middle middle box
    elif (2 < col <= 5) and (2 < row <= 5):
        col_indexes = [3,4,5]
        row_indexes = [3,4,5]

    #middle right box
    elif (5 < col <= 8) and (2 < row <= 5):
        col_indexes = [6,7,8]
        row_indexes = [3,4,5]

    #bottom left box
    elif (col <= 2) and (row > 5):
        col_indexes = [0,1,2]
        row_indexes = [6,7,8]

    #bottom middle box
    elif (2 < col <= 5) and (row > 5):
        col_indexes = [3,4,5]
        row_indexes = [6,7,8]

    #bottom right box
    elif (5 < col <= 8) and (row > 5):
        col_indexes = [6,7,8]
        row_indexes = [6,7,8]

    my_set = set()
    for i in row_indexes:
            for j in col_indexes:
                if sudoku_state[i][j] != '?':
                      my_set.add(sudoku_state[i][j])
    return my_set



print(get_square_values_back(parse_matrix(PUZZLE_PATH), 6, 1))

{'3', '5', '2', '1', '4'}


In [386]:

def get_unavailable_nums(sudoku_state, row, col):
    unavail_row = get_row_values_back(sudoku_state,row, col)
    unavail_col = get_col_values_back(sudoku_state,row, col)
    unavail_square = get_square_values_back(sudoku_state, row, col)
    my_set = unavail_row.union(unavail_col, unavail_square)
    return my_set

get_unavailable_nums(parse_matrix(PUZZLE_PATH),0,2)

{'1', '2', '4', '5', '6', '7', '8', '9', '?'}

# Genetic Algorithm Functions

In [ ]:
#planning
# penaty function (exogenous fitness) -> using the same method on simulated node for finding lowest
# replacement -> lets just do generational replacement, we can add steady state when we feel more confident
# Tournment selection style -> selecting individuals is probabalistic from population, but replacement is not
# Think about selection pressures, what are the probablities to select more fit verus less fit individuals
# we could take the average of the total conflicts each new population as testings


In [18]:
def get_average(current_generation_conflict_count):
    total_value = 0
    for value in range(len(current_generation_conflict_count)):
        total_value += current_generation_conflict_count[value]

    return total_value / len(current_generation_conflict_count)
print(get_average([8,9,10,4,5,7,3,6,2,8]))


def crossover(selected_parents):
    pass


def mutation(children_crossovers):
    pass





def genetic_algorithm_tournement(max_iterations):
    k = 0
    population_size = 0 #must be even size
    generational_conflict_count = [] # will typically be float

    current_population = create_population(population_size)#random k assignments with totally randomly generated states
    for generations in range(0, max_iterations):
        current_generation_conflict_count = []
        for individual in current_population:
            current_generation_conflict_count.append(individual.get_conflicts)
            if individual.get_conflicts == 0 or generations == (max_iterations-1):
                generation_conflict_average = get_average(current_generation_conflict_count)
                generational_conflict_count.append(generation_conflict_average)
                return individual, generational_conflict_count
        generational_conflict_count.append(get_average(current_generation_conflict_count))
        #we have made a new population and we have made sure our answer is not a part of it by this point
        selected_parents = tournement_selection(current_population, k) #returns array of selected parents to build next generation
        children_crossovers = crossover(selected_parents)# with the crossovers being again random
        mutated_children = mutation(children_crossovers)
        current_population = mutated_children
    return None

6.2


In [17]:
def create_population(population_size):
    population = []
    for individual in range(0,population_size):
       new_individual = ga_state_node(parse_matrix(PUZZLE_PATH))
       new_individual.fill_random_values()
       new_individual.process_total_conflicts()
       population.append(new_individual)
    return population

#test_population = create_population(5)
#for individual in test_population:
#    print(individual.get_conflict())

106
128
121
127
122


In [2]:
class ga_state_node():
    #overall much is borrowed from the SA state node layout, with some key changes for how the genetic algorithm works
    # the philosophy of most of these programs is keep lots of the functionally on the methods of the states, so as tied closer to the nodes, additionally
    # making the actual algorithm code cleaner and maintaining SOP
    def __init__(self, raw_matrix):
        self.matrix = raw_matrix
        self.num_conflict = -1

    def __str__(self):
        str_builder = ""
        for rows in self.matrix:
            str_builder += str(rows)
            str_builder += "\n"
        return str_builder

    def get_conflict(self):
        return self.num_conflict

    def process_total_conflicts(self):
        current_state_conflicts = 0
        for col_index in range(0, 9):
            for row_index in range(0, 9):
                if not isinstance(self.matrix[col_index][row_index], str):
                    current_state_conflicts += alldiff(self.matrix, row_index, col_index)
        self.num_conflict = current_state_conflicts

    # we are distinguishing our hardcoded values from our mutable values from the data type. in our filling random values, we only fill the question marks with integers
    # If we encounter a str type in our search, we skip it as it is a hardcoded value
    def fill_random_values(self):
        for col in range(len(self.matrix)):
            for row in range(len(self.matrix[col])):
                if self.matrix[col][row] == '?':
                    self.matrix[col][row] = int(random.randint(1, 9))
                    #fills question marks with random values

    def get_highest_conflict_value(self):
        #this method is essentially the minimum conflict heuristic added to our states as a method, we use this in conjunction with the the helper function alldiff to return the highest conflict square
        worse_value = -1
        worse_row_index = 0
        worse_col_index = 0
        for col_index in range(0, 9):
            for row_index in range(0, 9):
                if not isinstance(self.matrix[col_index][row_index], str):
                    conflict_position = alldiff(self.matrix, row_index, col_index)
                    if conflict_position > worse_value:
                        worse_value = conflict_position
                        worse_row_index = row_index
                        worse_col_index = col_index
        return worse_row_index, worse_col_index




In [ ]:
def tournement_selection(current_population, contestant_round_size):
    population_size = len(current_population)
    tournament_rounds = population_size // 2 #think about this??? we will only select half the population to return and the best
    tournament_winners = []
    for rounds in range(0, tournament_rounds):
        round_winner = None
        round_minimum_conflict = MAX_INT32
        round_contestants = []

        for uniform_pick in range(0, contestant_round_size):
            random.shuffle(current_population)
            round_contestants.append(current_population[0])

        for contestant in round_contestants:
            if contestant.get_conflicts() < round_minimum_conflict:
                round_winner = contestant
                round_minimum_conflict = contestant.get_conflicts()

        tournament_winners.append(round_winner)



# Simulated Annealing Algorithm Functions

In [ ]:
def schedule(time_iteration):
    #Arbitrary hyperparameter
    t0 = 120
    #our initial temp
    alpha = .995
    #perctage of cooling down per iteration

    if time_iteration < 1000:
        return t0 * (alpha ** time_iteration)
    elif time_iteration < 4000:
        return t0 * ((alpha-.0005) ** time_iteration)
    else:
        #exploration versus explotation, after a number of exploritory searches, we wanna start narrowing the search
        return t0 * ((alpha-.005) ** time_iteration)

    #ive tuned this a bunch, this should mean around 5% cooling per cycle


In [ ]:
class sa_state_node():
    def __init__(self, raw_matrix):
        self.matrix = raw_matrix
        self.num_conflicts = [-1]
        # how winning nodes do.
        self.children = []  # array of children that the given node has.
        #we want to immediately be able to know which indexes are hardcoded or not for a basic constraint

    def __str__(self):
        str_builder = ""
        for rows in self.matrix:
            str_builder += str(rows)
            str_builder += "\n"
        return str_builder

    def get_conflict(self):
        return int(self.num_conflicts[len(self.num_conflicts) - 1])

    def process_total_conflicts(self):
        current_state_conflicts = 0
        for col_index in range(0, 9):
            for row_index in range(0, 9):
                if not isinstance(self.matrix[col_index][row_index], str):
                    current_state_conflicts += alldiff(self.matrix, row_index, col_index)
        self.num_conflicts.append(current_state_conflicts)
        # a clever system to keep track of the conflicts per state could be used for post analysis as a metric to see how rapidly between

    # we are distinguishing our hardcoded values from our mutable values from the data type. in our filling random values, we only fill the question marks with integers
    # If we encounter a str type in our search, we skip it as it is a hardcoded value
    def fill_random_values(self):
        for col in range(len(self.matrix)):
            for row in range(len(self.matrix[col])):
                if self.matrix[col][row] == '?':
                    self.matrix[col][row] = int(random.randint(1, 9))
                    #fills question marks with random values

    def get_highest_conflict_value(self):
        #this method is essentially the minimum conflict heuristic added to our states as a method, we use this in conjunction with the the helper function alldiff to return the highest conflict square
        worse_value = -1
        worse_row_index = 0
        worse_col_index = 0
        for col_index in range(0, 9):
            for row_index in range(0, 9):
                if not isinstance(self.matrix[col_index][row_index], str):
                    conflict_position = alldiff(self.matrix, row_index, col_index)
                    if conflict_position > worse_value:
                        worse_value = conflict_position
                        worse_row_index = row_index
                        worse_col_index = col_index
        return worse_row_index, worse_col_index



    def generate_neighbor(self):
        percent_worse_conflict_choice = .2
        #this hyperparameter is directly related to our minimum conflict heuristic, enabling us to choose the percentage that a the worst value gets picked to change
        # versus picking a completely random value to change. Allowing a half and half min conflict heuristic still with some ability to diversity
        while True:
            rand_x = random.randint(0, 8)
            rand_y = random.randint(0, 8)
            print(self.matrix)
            if not isinstance(self.matrix[rand_y][rand_x], str):
                #was having problems, this make an actual copy to give to the next state, and not the reference like I had before
                new_neighbor_matrix = []
                for row in self.matrix:
                    new_row = []
                    for cell in row:
                        new_row.append(cell)
                    new_neighbor_matrix.append(new_row)
                #this is a small optimization strategy of picking the most conflicting space to change randomly
                new_neighbor_node = sa_state_node(new_neighbor_matrix)
                #we just grab a percentage chance against our additional hyperparameter
                random_choice = random.random()
                if random_choice < percent_worse_conflict_choice:
                    new_neighbor_node.matrix[rand_y][rand_x] = random.randint(1, 9)
                else:
                    worst_x, worst_y = new_neighbor_node.get_highest_conflict_value()
                    new_neighbor_node.matrix[worst_y][worst_x] = random.randint(1, 9)

                break

        return new_neighbor_node


#Testing, you can take a look at how we are generating the matricies
test = sa_state_node(parse_matrix(PUZZLE_PATH))
test.process_total_conflicts()
print(test.get_conflict())

In [3]:
def probability(delta_e, T):
    #this could be boltzman constant but we are using one for now
    if T == 0:
       return False
    try:
        chance = math.exp(-delta_e / (1 * T))
        #1 is our k, i switch it out with boltzman constant to see differences
        return random.random() < chance
    except OverflowError:
        #needed as a just in case
        return False


def simulated_annealing(max_iterations):
    conflict_history = []
    #global history varible to see how conflicts are changed over time super useful for testing
    current = sa_state_node(parse_matrix(PUZZLE_PATH))
    #initialised state/node with puzzle input filled in random values.
    current.fill_random_values()
    current.num_conflicts = conflict_history
    current.process_total_conflicts()

    for time_iteration in range(max_iterations):
        T = schedule(time_iteration)
        if T == 0 or current.get_conflict() == 0:
            return current
        current_conflicts = current.get_conflict()
        next = current.generate_neighbor()
        next.num_conflicts = conflict_history
        next.process_total_conflicts()
        next_conflicts = next.get_conflict()
        delta_e = next_conflicts - current_conflicts
        if delta_e < 0:
            #critial, we want negitive change!! we are minimizing conflicts over generations
            current = next
        else:
            if probability(delta_e, T):
                current = next
    return current


#plt.ylabel("Number of Total Conflicts In State")
#plt.xlabel("Number of Iterations")
#plt.plot(winning_node.num_conflicts)

# Backtracking Algorithm Functions

In [411]:

def is_available(sudoku_state, row, col, num):
    # uses get_unavailable_nums to return what nums are there already
    #then just returns true or false if num is available
    my_list = get_unavailable_nums(sudoku_state,row,col)
    if str(num) in my_list:
        return False
    else:
        return True


def solver_back(sudoku_state, row, col):
    #if sudoku board is complete
    if row == 8 and col == 9:
        return True

    # If last column of the row go to the next row
    if col == 9:
        row += 1
        col = 0

    # If spot is already filled then go to next
    if sudoku_state[row][col] != '?':
        return solver_back(sudoku_state, row, col+1)

    for num in range(1, 10):
        # If num is available in the current spot add it
        if is_available(sudoku_state, row, col, num):
            sudoku_state[row][col] = str(num)
            #TODO: add global counter for num_steps
            if solver_back(sudoku_state, row, col+1):
                return True
            sudoku_state[row][col] = '?'
    #TODO: add global counter for num_backtracks
    return False

def solveSudoku_back(sudoku_state): #starts recursive solving
    solver_back(sudoku_state, 0, 0)
    for line in sudoku_state:
        print(line)

solveSudoku_back(parse_matrix(PUZZLE_PATH))

['1', '9', '8', '7', '5', '6', '2', '4', '3']
['7', '5', '4', '2', '8', '3', '6', '1', '9']
['2', '6', '3', '1', '9', '4', '8', '5', '7']
['6', '1', '7', '4', '2', '9', '5', '3', '8']
['3', '4', '9', '5', '6', '8', '1', '7', '2']
['5', '8', '2', '3', '1', '7', '9', '6', '4']
['8', '3', '5', '6', '4', '2', '7', '9', '1']
['4', '2', '1', '9', '7', '5', '3', '8', '6']
['9', '7', '6', '8', '3', '1', '4', '2', '5']


# Forward Checking Algorithm Functions

In [388]:

def available(sudoku_state, row, col):
    unavail_list = get_unavailable_nums(sudoku_state,row,col)
    my_set = {'1', '2', '3', '4', '5', '6', '7', '8', '9'}
    my_set = my_set.difference(unavail_list)
    return my_set


def solver_for(sudoku_state, row, col):
    #if sudoku board is complete
    if row == 8 and col == 9:
        return True

    # If last column of the row go to the next row
    if col == 9:
        row += 1
        col = 0

    # If spot is already filled then go to next
    if sudoku_state[row][col] != '?':
        return solver_for(sudoku_state, row, col+1)

    #create domain of available numbers
    avail_list = available(sudoku_state, row, col)
    #test each available numbers recursively like normal back tracking
    for num in avail_list:
        sudoku_state[row][col] = str(num)
        #TODO ADD COUNTER HERE FOR EACH STEP
        if solver_for(sudoku_state, row, col+1):
            return True
        sudoku_state[row][col] = '?'
    #TODO: add num_backtracking counter
    return False

def solveSudoku_forward(sudoku_state): #starts recursive solving
    solver_for(sudoku_state, 0, 0)
    for line in sudoku_state:
        pass
        print(line)

solveSudoku_forward(parse_matrix(PUZZLE_PATH))

['1', '9', '8', '7', '5', '6', '2', '4', '3']
['7', '5', '4', '2', '8', '3', '6', '1', '9']
['2', '6', '3', '1', '9', '4', '8', '5', '7']
['6', '1', '7', '4', '2', '9', '5', '3', '8']
['3', '4', '9', '5', '6', '8', '1', '7', '2']
['5', '8', '2', '3', '1', '7', '9', '6', '4']
['8', '3', '5', '6', '4', '2', '7', '9', '1']
['4', '2', '1', '9', '7', '5', '3', '8', '6']
['9', '7', '6', '8', '3', '1', '4', '2', '5']


# Arc Consistancy Algorithm Functions

In [429]:
#TODO: implement Arc Consistency!!!!!!!!!!!!!!!!!!!!!!

def AvailableDomain(sudoku_state, row, col):
    unavail_list = get_unavailable_nums(sudoku_state,row,col)
    my_set = {'1', '2', '3', '4', '5', '6', '7', '8', '9'}
    my_set = my_set.difference(unavail_list)
    return my_set

def remove_inc(queue, current, domains):
    pass
    # removed = False
    # for x in set(domains[]):
    #     if len(domains[x]) == 0:
    #         domains.remove(x)
    #         removed = True
    # return removed

def Ac3(domains):
    pass
    # queue = []
    # while len(queue) != 0:
    #     current = queue.pop(0)
    #     if remove_inc(queue, current, domains):
    #         if len(domains) !=0:
    #             for k in neighbors():
    #                 queue.append(k)
    #
    #             return True
    #         return False


def solver_arc(sudoku_state, row, col, domains):
    #if sudoku board is complete
    if row == 8 and col == 9:
        return True

    # If last column of the row go to the next row
    if col == 9:
        row += 1
        col = 0

    # If spot is already filled then go to next
    if sudoku_state[row][col] != '?':
        return solver_arc(sudoku_state, row, col+1)

    #create domain of available numbers
    avail_list = list(domains[(row,col)])
    print(avail_list)
    print(domains)
    #test each available numbers recursively like normal back tracking
    for num in avail_list:
        #TODO: add global counter for num_steps
        sudoku_state[row][col] = str(num)
        if Ac3(domains):
            if solver_arc(sudoku_state, row, col+1, domains):
                return True
        sudoku_state[row][col] = '?'
    #Todo: add global counter for num_backtracks
    return False

def solveSudoku_arc(sudoku_state): #starts recursive solving
    domains = {}
    for row in range(9):
        for col in range(9):
            if sudoku_state[row][col] == '?':
                domains[row,col] = available(sudoku_state, row, col)
    Ac3(domains)
    solver_arc(sudoku_state, 0, 0, domains)
    for line in sudoku_state:
        pass
        # print(line)

solveSudoku_arc(parse_matrix(PUZZLE_PATH))


['2', '3', '9', '1']
{(0, 0): {'2', '3', '9', '1'}, (0, 1): {'1', '9'}, (0, 3): {'9', '2', '1', '4', '7'}, (0, 6): {'2', '7'}, (0, 7): {'4', '2', '3', '7'}, (0, 8): {'2', '3', '7'}, (1, 1): {'5'}, (1, 3): {'2'}, (1, 4): {'8'}, (1, 5): {'2', '3', '8'}, (2, 0): {'2', '3', '9', '1'}, (2, 1): {'6', '1', '9'}, (2, 2): {'6', '3'}, (2, 3): {'9', '2', '1', '4', '7'}, (2, 4): {'4', '9', '7'}, (2, 5): {'3', '2', '1', '4', '7'}, (2, 8): {'2', '3', '7'}, (3, 1): {'4', '8', '1'}, (3, 3): {'4'}, (3, 7): {'8', '3'}, (3, 8): {'8', '3'}, (4, 0): {'8', '3', '5'}, (4, 1): {'4', '8', '5'}, (4, 3): {'7', '4', '5'}, (4, 5): {'7', '4', '8', '5'}, (4, 7): {'7', '2', '3', '8'}, (4, 8): {'7', '2', '3', '8'}, (5, 0): {'8', '5'}, (5, 1): {'8', '5'}, (5, 5): {'7', '8', '5'}, (5, 7): {'6', '8', '7'}, (6, 0): {'8', '9'}, (6, 3): {'9', '6', '2', '1', '4', '7'}, (6, 4): {'4', '9', '7'}, (6, 5): {'4', '2', '1', '7'}, (6, 6): {'2', '7'}, (6, 7): {'7', '2', '9', '8'}, (6, 8): {'7', '2', '1', '8'}, (7, 3): {'7', '9', '5'}